# <center>**Project: `London Crime Analysis`**

**AUTHOR:** `Damian Ferguson`<br>
**CREATED:** `11 Nov 2024`<br>
**LAST UPDATED:** `28 Jan 2025`<br>

This notebook is used to prepare London crime and population data for exploratory analysis in a Tableau dashboard.  Data is imported, inspected, cleaned, validated and simplified.  The clean data is then saved to new files ready for use in Tableau.

**SOURCE DATA:**<br>
`UK Government` <br>
Land area and population density figures for 2001 to 2050 for boroughs. <br>
File(s): housing-density-borough.csv <br>
https://www.data.gov.uk/dataset/a76f46f9-c10b-4fe7-82f6-aa928471fcd1/land-area-and-population-density-ward-and-borough <br>
<br>

`Metropolitan Police Service`<br>
The number of crimes at borough levels for London per month, according to crime type. Two files covering the last available 24 months only and all historic for full calendar years. <br>
File(s): MPS Borough Level Crime (most recent 24 months).csv | MPS Borough Level Crime (Historical).csv  <br>
https://data.london.gov.uk/dataset/recorded_crime_summary <br>
<br>

**OUTPUT DATA:**<br>
- `tab_borough_list.csv` contains a list of unique London borough codes and borough names
- `tab_borough_population.csv` contains breakdown of population by borough and year
- `tab_yearly_crime.csv` contains breakdown of crime by borough, year, major and minor crime categories

# **Set Up Environment**

In [5]:
# Import core libraries
import numpy as np
import pandas as pd

# Import additional libraries
from datetime import datetime

In [7]:
# Reusable functions
def diff_between_sets(series1, series2):
    """
    Display items in Set 1 that are not in Set 2 and
    items in Set 2 not in Set 1
    """
    set1 = set(series1.unique())
    set2 = set(series2.unique())
    print(f"In 1, Not In 2: {set1.difference(set2)}")
    print(f"In 2, Not In 1: {set2.difference(set1)}")

# **Data Ingestion**

## **London Boroughs**

### **Import and Inspect Data**

In [12]:
# Load data to dataframe.
boroughs = pd.read_csv("housing-density-borough.csv")

# View metatdata and first five rows of data.
print(boroughs.info())
boroughs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Code                             1872 non-null   object 
 1   Name                             1872 non-null   object 
 2   Year                             1872 non-null   int64  
 3   Source                           1872 non-null   object 
 4   Population                       1872 non-null   int64  
 5   Inland_Area _Hectares            1872 non-null   float64
 6   Total_Area_Hectares              1872 non-null   float64
 7   Population_per_hectare           1872 non-null   float64
 8   Square_Kilometres                1872 non-null   float64
 9   Population_per_square_kilometre  1872 non-null   float64
dtypes: float64(5), int64(2), object(3)
memory usage: 146.4+ KB
None


,Code,Name,Year,Source,Population,Inland_Area _Hectares,Total_Area_Hectares,Population_per_hectare,Square_Kilometres,Population_per_square_kilometre
0,E09000001,City of London,1999,ONS MYE,6581,290.4,314.9,22.7,2.9,2266.2
1,E09000001,City of London,2000,ONS MYE,7014,290.4,314.9,24.2,2.9,2415.3
2,E09000001,City of London,2001,ONS MYE,7359,290.4,314.9,25.3,2.9,2534.1
3,E09000001,City of London,2002,ONS MYE,7280,290.4,314.9,25.1,2.9,2506.9
4,E09000001,City of London,2003,ONS MYE,7115,290.4,314.9,24.5,2.9,2450.1


In [14]:
# Delete unwanted columns.
boroughs.drop(columns = ["Source",
                         "Inland_Area _Hectares",
                         "Total_Area_Hectares",
                         "Population_per_hectare"],
                         inplace = True)

# Rename columns
boroughs.rename(columns = {"Code":                           "borough_code",
                          "Name":                            "borough_name",
                          "Year":                            "year",
                          "Population":                      "population",
                          "Square_Kilometres":               "sqkms",
                          "Population_per_square_kilometre": "population_sqkm"},
               inplace = True)

# Check changes successful.
print(boroughs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   borough_code     1872 non-null   object 
 1   borough_name     1872 non-null   object 
 2   year             1872 non-null   int64  
 3   population       1872 non-null   int64  
 4   sqkms            1872 non-null   float64
 5   population_sqkm  1872 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 87.9+ KB
None


### **Validate Boroughs**

In [17]:
# Number of unique borough codes ... should be 33
len(boroughs['borough_code'].unique())

36

In [19]:
# View unique borough codes ... should be 33
boroughs[['borough_code', 'borough_name']].groupby(['borough_code', 'borough_name']).count()

,
borough_code,borough_name
E09000001,City of London
E09000002,Barking and Dagenham
E09000003,Barnet
E09000004,Bexley
E09000005,Brent
E09000006,Bromley
E09000007,Camden
E09000008,Croydon
E09000009,Ealing


#### **Remove Summarised Data from Boroughs**

In [22]:
# Remove borough codes E12000007, E13000001 and E13000002 as these
# records contain summarised data not borough level data

# View unique boroughs before change.
print("Boroughs codes before change:\n", boroughs["borough_code"].describe())
print(boroughs["borough_code"].unique())

boroughs = boroughs [(boroughs["borough_code"] != "E12000007") &
                     (boroughs["borough_code"] != "E13000001") &
                     (boroughs["borough_code"] != "E13000002")]

# Check changes successful.
print("\nBoroughs codes after change:\n", boroughs["borough_code"].describe())
print(boroughs["borough_code"].unique())

Boroughs codes before change:
 count          1872
unique           36
top       E09000001
freq             52
Name: borough_code, dtype: object
['E09000001' 'E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006'
 'E09000007' 'E09000008' 'E09000009' 'E09000010' 'E09000011' 'E09000012'
 'E09000013' 'E09000014' 'E09000015' 'E09000016' 'E09000017' 'E09000018'
 'E09000019' 'E09000020' 'E09000021' 'E09000022' 'E09000023' 'E09000024'
 'E09000025' 'E09000026' 'E09000027' 'E09000028' 'E09000029' 'E09000030'
 'E09000031' 'E09000032' 'E09000033' 'E12000007' 'E13000001' 'E13000002']

Boroughs codes after change:
 count          1716
unique           33
top       E09000001
freq             52
Name: borough_code, dtype: object
['E09000001' 'E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006'
 'E09000007' 'E09000008' 'E09000009' 'E09000010' 'E09000011' 'E09000012'
 'E09000013' 'E09000014' 'E09000015' 'E09000016' 'E09000017' 'E09000018'
 'E09000019' 'E09000020' 'E09000021' 'E09000022' 'E0

### **Check for Missing and Duplicated Data**

In [25]:
# Check for missing data.
print("Check for missing data:\n", boroughs.isna().sum())

# Check for duplicate values.
print("\nCheck for duplicate values:\n", boroughs.duplicated().value_counts())

Check for missing data:
 borough_code       0
borough_name       0
year               0
population         0
sqkms              0
population_sqkm    0
dtype: int64

Check for duplicate values:
 False    1716
Name: count, dtype: int64


### **Align Years in Boroughs with Years in Crime Data (2011-23 inclusive)**

In [28]:
# View unique years.
boroughs['year'].unique()

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031,
       2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042,
       2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050])

In [30]:
# Keep 2011 to 2023 data, delete all other rows.
boroughs = boroughs [(boroughs["year"] > 2010) & (boroughs["year"] < 2024)]

# Check changes successful.
print(boroughs["year"].unique())

[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [32]:
# Create a list of unnique borough codes and names for validation
# and future reference.
borough_list = boroughs[['borough_code', 'borough_name']].groupby(['borough_code', 'borough_name']).count().reset_index()
borough_list

,borough_code,borough_name
0,E09000001,City of London
1,E09000002,Barking and Dagenham
2,E09000003,Barnet
3,E09000004,Bexley
4,E09000005,Brent
5,E09000006,Bromley
6,E09000007,Camden
7,E09000008,Croydon
8,E09000009,Ealing
9,E09000010,Enfield


## **MPS Recorded Crime**

### **Import Recent and Historic Data**

In [36]:
# Import the recent data
recent_df = pd.read_csv('MPS Borough Level Crime (most recent 24 months).csv')

# Change columns names
recent_df.rename(columns = {'MajorText': 'major_cat',
                            'MinorText': 'minor_cat',
                            'BoroughName': 'borough_name'},
                inplace = True)

# View data and data types
print(recent_df.info())
recent_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   major_cat     1015 non-null   object
 1   minor_cat     1015 non-null   object
 2   borough_name  1015 non-null   object
 3   202209        1015 non-null   int64 
 4   202210        1015 non-null   int64 
 5   202211        1015 non-null   int64 
 6   202212        1015 non-null   int64 
 7   202301        1015 non-null   int64 
 8   202302        1015 non-null   int64 
 9   202303        1015 non-null   int64 
 10  202304        1015 non-null   int64 
 11  202305        1015 non-null   int64 
 12  202306        1015 non-null   int64 
 13  202307        1015 non-null   int64 
 14  202308        1015 non-null   int64 
 15  202309        1015 non-null   int64 
 16  202310        1015 non-null   int64 
 17  202311        1015 non-null   int64 
 18  202312        1015 non-null   int64 
 19  202401

,major_cat,minor_cat,borough_name,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,4,4,3,1,3,3,2,...,3,4,4,5,6,3,8,2,3,9
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Barking and Dagenham,100,138,101,91,93,91,110,...,89,125,127,124,131,110,99,104,131,106
2,BURGLARY,BURGLARY - RESIDENTIAL,Barking and Dagenham,0,0,0,0,0,0,0,...,0,0,0,2,50,52,40,37,38,43
3,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,29,25,21,21,27,18,32,...,19,20,31,21,30,24,28,32,22,25
4,BURGLARY,BURGLARY IN A DWELLING,Barking and Dagenham,75,78,64,84,91,38,55,...,64,53,67,47,0,0,0,0,0,0


In [38]:
# Import the historic data
historic_df = pd.read_csv('MPS Borough Level Crime (Historical).csv')

# Change columns names
historic_df.rename(columns = {'MajorText': 'major_cat',
                            'MinorText': 'minor_cat',
                            'BoroughName': 'borough_name'},
                inplace = True)

# Check change worked
print(historic_df.info())
historic_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Columns: 152 entries, major_cat to 202208
dtypes: int64(149), object(3)
memory usage: 1.1+ MB
None


,major_cat,minor_cat,borough_name,201004,201005,201006,201007,201008,201009,201010,...,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208
0,ARSON AND CRIMINAL DAMAGE,ARSON,Barking and Dagenham,6,5,11,10,6,6,13,...,5,1,3,3,2,5,6,5,9,5
1,ARSON AND CRIMINAL DAMAGE,CRIMINAL DAMAGE,Barking and Dagenham,208,190,218,217,203,161,196,...,105,114,123,107,111,124,111,136,112,127
2,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,49,58,58,46,46,51,53,...,20,14,18,28,27,15,15,23,18,27
3,BURGLARY,BURGLARY IN A DWELLING,Barking and Dagenham,118,102,124,137,153,136,116,...,82,92,78,65,73,79,70,51,44,72
4,DRUG OFFENCES,POSSESSION OF DRUGS,Barking and Dagenham,76,64,82,72,98,87,109,...,112,86,114,124,127,70,106,118,116,108


### **Check and Align Recent and Historic Crime Categories**

In [41]:
# Identify values in recent data not in historic and vice versa
print("MAJOR CATEGORY")
diff_between_sets(recent_df['major_cat'], historic_df['major_cat'])

print("MINOR CATEGORY")
diff_between_sets(recent_df['minor_cat'], historic_df['minor_cat']) 

print("BOROUGH NAME")
diff_between_sets(recent_df['borough_name'], historic_df['borough_name'])   

MAJOR CATEGORY
In 1, Not In 2: {'NFIB FRAUD'}
In 2, Not In 1: set()
MINOR CATEGORY
In 1, Not In 2: {'BURGLARY NON-DWELLING', 'NFIB', 'BURGLARY - RESIDENTIAL'}
In 2, Not In 1: set()
BOROUGH NAME
In 1, Not In 2: set()
In 2, Not In 1: set()


#### **Remove Major Category** `NFIB FRAUD`
Major Category = NFIB FRAUD, Minor Category = NFIB<br><br>
This major/minor category appears in the `recent` data set but not the `historic` data set.  There are 12 rows of data and a total of 17 offences recorded.  The National Fraud Intelligence Bureau (NFIB) is a UK police unit that analyzes and gathers intelligence on fraud and cyber crime.  As NFIB is a national crime unit, remove this data from the `recent` data set.

Source(s):<br> 
https://data.london.gov.uk/dataset/recorded_crime_summary <br>
https://en.wikipedia.org/wiki/National_Fraud_Intelligence_Bureau

In [44]:
# Remove NFIB Fraud data from recent data set
recent_df = recent_df [recent_df['major_cat'] != "NFIB FRAUD"]

# Confirm change
recent_df['major_cat'].unique()

array(['ARSON AND CRIMINAL DAMAGE', 'BURGLARY', 'DRUG OFFENCES',
       'FRAUD AND FORGERY', 'MISCELLANEOUS CRIMES AGAINST SOCIETY',
       'POSSESSION OF WEAPONS', 'PUBLIC ORDER OFFENCES', 'ROBBERY',
       'SEXUAL OFFENCES', 'THEFT', 'VEHICLE OFFENCES',
       'VIOLENCE AGAINST THE PERSON'], dtype=object)

#### **Consolidate Minor Categories Within** `BURGLARY` **Major Category**
Major Category = BURGLARY, Minor Category = BURGLARY NON-DWELLING and BURGLARY - RESIDENTIAL<br><br>
In the `recent` data set, burglary has four minor categories; the `historic` data set has two.  This is a change to the way data is recorded by the MPS/Home Office, therefore will re-map `recent` data set (four minor categories) to align with `historic` data set (two minor categories.

BURGLARY IN A DWELLING = BURGLARY IN A DWELLING + BURGLARY - RESIDENTIAL

BURGLARY BUSINESS AND COMMUNITY = BURGLARY BUSINESS AND COMMUNITY + BURGLARY NON-DWELLING

Source(s):<br> 
https://data.london.gov.uk/dataset/recorded_crime_summary

In [47]:
# Confirm minor categories for burglary in historic data set
print(historic_df[['minor_cat']] [historic_df['major_cat'] == "BURGLARY"]\
      .groupby( 'minor_cat').size())

# Create a subset of recent data with major category of burglary only
r_burglary = recent_df [recent_df['major_cat'] == "BURGLARY"]

# Check data is burglary only
print(r_burglary['major_cat'].unique())
print(r_burglary['minor_cat'].unique())

minor_cat
BURGLARY BUSINESS AND COMMUNITY    33
BURGLARY IN A DWELLING             33
dtype: int64
['BURGLARY']
['BURGLARY - RESIDENTIAL' 'BURGLARY BUSINESS AND COMMUNITY'
 'BURGLARY IN A DWELLING' 'BURGLARY NON-DWELLING']


In [49]:
# Check data before combining
recent_df [(recent_df['minor_cat'] == 'BURGLARY IN A DWELLING') |\
    (recent_df['minor_cat'] == 'BURGLARY - RESIDENTIAL')].head(6)

,major_cat,minor_cat,borough_name,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
2,BURGLARY,BURGLARY - RESIDENTIAL,Barking and Dagenham,0,0,0,0,0,0,0,...,0,0,0,2,50,52,40,37,38,43
4,BURGLARY,BURGLARY IN A DWELLING,Barking and Dagenham,75,78,64,84,91,38,55,...,64,53,67,47,0,0,0,0,0,0
31,BURGLARY,BURGLARY - RESIDENTIAL,Barnet,0,0,0,0,0,0,0,...,0,0,0,15,166,158,131,114,122,131
33,BURGLARY,BURGLARY IN A DWELLING,Barnet,140,199,178,172,192,140,180,...,166,165,166,133,0,0,0,0,0,0
61,BURGLARY,BURGLARY - RESIDENTIAL,Bexley,0,0,0,0,0,0,0,...,0,0,0,2,52,49,37,30,32,36
63,BURGLARY,BURGLARY IN A DWELLING,Bexley,40,72,52,49,37,47,71,...,71,85,84,54,0,0,0,0,0,0


In [51]:
# Create a subset of data combing BURGLARY - RESIDENTIAL and BURGLARY IN A DWELLING
# and set minor category to BURGLARY IN A DWELLING for all rows of data
r_burg1 = recent_df [(recent_df['minor_cat'] == 'BURGLARY IN A DWELLING') |\
    (recent_df['minor_cat'] == 'BURGLARY - RESIDENTIAL')]\
    .groupby(['borough_name', 'major_cat'])\
    .sum()\
    .reset_index()

r_burg1['minor_cat'] = 'BURGLARY IN A DWELLING'
r_burg1.head(3)

,borough_name,major_cat,minor_cat,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
0,Barking and Dagenham,BURGLARY,BURGLARY IN A DWELLING,75,78,64,84,91,38,55,...,64,53,67,49,50,52,40,37,38,43
1,Barnet,BURGLARY,BURGLARY IN A DWELLING,140,199,178,172,192,140,180,...,166,165,166,148,166,158,131,114,122,131
2,Bexley,BURGLARY,BURGLARY IN A DWELLING,40,72,52,49,37,47,71,...,71,85,84,56,52,49,37,30,32,36


In [53]:
# Check data before combining
recent_df [(recent_df['minor_cat'] == 'BURGLARY BUSINESS AND COMMUNITY') |\
    (recent_df['minor_cat'] == 'BURGLARY NON-DWELLING')].head(6)

,major_cat,minor_cat,borough_name,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
3,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barking and Dagenham,29,25,21,21,27,18,32,...,19,20,31,21,30,24,28,32,22,25
32,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Barnet,53,50,65,55,53,52,50,...,65,49,55,65,62,37,37,46,53,32
34,BURGLARY,BURGLARY NON-DWELLING,Barnet,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
62,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Bexley,19,20,28,12,23,17,15,...,28,19,32,25,30,17,16,21,15,11
90,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Brent,40,40,47,52,46,53,44,...,39,52,54,56,65,35,49,45,65,43
92,BURGLARY,BURGLARY NON-DWELLING,Brent,0,0,0,0,0,0,0,...,0,0,0,0,2,1,1,0,1,1


In [55]:
# Create a subset of data combing BURGLARY BUSINESS AND COMMUNITY and
# BURGLARY NON-DWELLING and set minor category to BURGLARY BUSINESS AND
# COMMUNITY for all rows of data
r_burg2 = recent_df [(recent_df['minor_cat'] == 'BURGLARY BUSINESS AND COMMUNITY') |\
    (recent_df['minor_cat'] == 'BURGLARY NON-DWELLING')]\
    .groupby(['borough_name', 'major_cat'])\
    .sum()\
    .reset_index()

r_burg2['minor_cat'] = 'BURGLARY BUSINESS AND COMMUNITY'
r_burg2.head(4)

,borough_name,major_cat,minor_cat,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
0,Barking and Dagenham,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,29,25,21,21,27,18,32,...,19,20,31,21,30,24,28,32,22,25
1,Barnet,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,53,50,65,55,53,52,50,...,65,49,55,65,63,37,37,46,53,32
2,Bexley,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,19,20,28,12,23,17,15,...,28,19,32,25,30,17,16,21,15,11
3,Brent,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,40,40,47,52,46,53,44,...,39,52,54,56,67,36,50,45,66,44


#### **Merge New Burglary Subsets with Original Data**

In [58]:
# Combine the two burglary subsets
r_burg3 = pd.concat([r_burg1, r_burg2])
r_burg3

,borough_name,major_cat,minor_cat,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
0,Barking and Dagenham,BURGLARY,BURGLARY IN A DWELLING,75,78,64,84,91,38,55,...,64,53,67,49,50,52,40,37,38,43
1,Barnet,BURGLARY,BURGLARY IN A DWELLING,140,199,178,172,192,140,180,...,166,165,166,148,166,158,131,114,122,131
2,Bexley,BURGLARY,BURGLARY IN A DWELLING,40,72,52,49,37,47,71,...,71,85,84,56,52,49,37,30,32,36
3,Brent,BURGLARY,BURGLARY IN A DWELLING,122,98,138,105,144,116,111,...,151,135,127,151,143,102,109,109,115,93
4,Bromley,BURGLARY,BURGLARY IN A DWELLING,86,91,107,94,86,83,112,...,99,110,97,97,88,79,50,67,57,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,Tower Hamlets,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,58,58,74,56,138,82,124,...,65,78,98,65,80,76,81,85,91,75
30,Unknown,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,46,56,52,38,66,42,92,...,6,9,14,10,38,44,43,24,55,38
31,Waltham Forest,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,27,40,23,29,43,42,36,...,53,51,35,43,39,50,58,57,74,64
32,Wandsworth,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,39,48,42,56,48,66,50,...,49,36,45,43,50,33,53,40,53,58


In [60]:
# Remove BURGLARY from recent data
recent_df = recent_df [recent_df['major_cat'] != 'BURGLARY']

# Check it was removed
print(recent_df['major_cat'].unique())

# Combine with new BURGLARY data set
recent_df = pd.concat([recent_df, r_burg3])

# Check it worked
print(recent_df['major_cat'].unique())
recent_df [recent_df['major_cat'] == 'BURGLARY']

['ARSON AND CRIMINAL DAMAGE' 'DRUG OFFENCES' 'FRAUD AND FORGERY'
 'MISCELLANEOUS CRIMES AGAINST SOCIETY' 'POSSESSION OF WEAPONS'
 'PUBLIC ORDER OFFENCES' 'ROBBERY' 'SEXUAL OFFENCES' 'THEFT'
 'VEHICLE OFFENCES' 'VIOLENCE AGAINST THE PERSON']
['ARSON AND CRIMINAL DAMAGE' 'DRUG OFFENCES' 'FRAUD AND FORGERY'
 'MISCELLANEOUS CRIMES AGAINST SOCIETY' 'POSSESSION OF WEAPONS'
 'PUBLIC ORDER OFFENCES' 'ROBBERY' 'SEXUAL OFFENCES' 'THEFT'
 'VEHICLE OFFENCES' 'VIOLENCE AGAINST THE PERSON' 'BURGLARY']


,major_cat,minor_cat,borough_name,202209,202210,202211,202212,202301,202302,202303,...,202311,202312,202401,202402,202403,202404,202405,202406,202407,202408
0,BURGLARY,BURGLARY IN A DWELLING,Barking and Dagenham,75,78,64,84,91,38,55,...,64,53,67,49,50,52,40,37,38,43
1,BURGLARY,BURGLARY IN A DWELLING,Barnet,140,199,178,172,192,140,180,...,166,165,166,148,166,158,131,114,122,131
2,BURGLARY,BURGLARY IN A DWELLING,Bexley,40,72,52,49,37,47,71,...,71,85,84,56,52,49,37,30,32,36
3,BURGLARY,BURGLARY IN A DWELLING,Brent,122,98,138,105,144,116,111,...,151,135,127,151,143,102,109,109,115,93
4,BURGLARY,BURGLARY IN A DWELLING,Bromley,86,91,107,94,86,83,112,...,99,110,97,97,88,79,50,67,57,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Tower Hamlets,58,58,74,56,138,82,124,...,65,78,98,65,80,76,81,85,91,75
30,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Unknown,46,56,52,38,66,42,92,...,6,9,14,10,38,44,43,24,55,38
31,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Waltham Forest,27,40,23,29,43,42,36,...,53,51,35,43,39,50,58,57,74,64
32,BURGLARY,BURGLARY BUSINESS AND COMMUNITY,Wandsworth,39,48,42,56,48,66,50,...,49,36,45,43,50,33,53,40,53,58


### **Convert Upper Case to Mixed Case**

In [63]:
# Convert text in major and minor category from upper case to title
# format (mixed case) for improved readability.
recent_df['major_cat'] = recent_df['major_cat'].str.title()
historic_df['major_cat'] = historic_df['major_cat'].str.title()

recent_df['minor_cat'] = recent_df['minor_cat'].str.title()
historic_df['minor_cat'] = historic_df['minor_cat'].str.title()

### **Create Consolidated Long-Form Data Set**

In [66]:
# Create long form historic data set
h_long = pd.melt(historic_df, id_vars = ['borough_name', 'major_cat', 'minor_cat'],
                 var_name = 'year_month', value_name = 'num_crimes')

h_long.head()

,borough_name,major_cat,minor_cat,year_month,num_crimes
0,Barking and Dagenham,Arson And Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson And Criminal Damage,Criminal Damage,201004,208
2,Barking and Dagenham,Burglary,Burglary Business And Community,201004,49
3,Barking and Dagenham,Burglary,Burglary In A Dwelling,201004,118
4,Barking and Dagenham,Drug Offences,Possession Of Drugs,201004,76


In [68]:
# Create long form recent data set
r_long = pd.melt(recent_df, id_vars = ['borough_name', 'major_cat', 'minor_cat'],
                 var_name = 'year_month', value_name = 'num_crimes')

r_long.head()

,borough_name,major_cat,minor_cat,year_month,num_crimes
0,Barking and Dagenham,Arson And Criminal Damage,Arson,202209,4
1,Barking and Dagenham,Arson And Criminal Damage,Criminal Damage,202209,100
2,Barking and Dagenham,Drug Offences,Possession Of Drugs,202209,77
3,Barking and Dagenham,Drug Offences,Trafficking Of Drugs,202209,9
4,Barking and Dagenham,Fraud And Forgery,Fraud And Forgery,202209,0


In [70]:
# Consolidate long form data sets
consolidated_df = pd.concat([h_long, r_long]).sort_values(['borough_name', 'major_cat', 'minor_cat', 'year_month']).reset_index(drop = True)

consolidated_df.head()

,borough_name,major_cat,minor_cat,year_month,num_crimes
0,Barking and Dagenham,Arson And Criminal Damage,Arson,201004,6
1,Barking and Dagenham,Arson And Criminal Damage,Arson,201005,5
2,Barking and Dagenham,Arson And Criminal Damage,Arson,201006,11
3,Barking and Dagenham,Arson And Criminal Damage,Arson,201007,10
4,Barking and Dagenham,Arson And Criminal Damage,Arson,201008,6


In [72]:
# Replace any infs with nan
consolidated_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check for missing values
consolidated_df.isna().sum()

borough_name    0
major_cat       0
minor_cat       0
year_month      0
num_crimes      0
dtype: int64

In [74]:
# Convert data object to datetime period fo data manipulation
consolidated_df['year_month'] = pd.to_datetime(consolidated_df['year_month'], format='%Y%m').dt.to_period('M')

# Check range of dates in year_month
print(f"Min month = {consolidated_df['year_month'].min()}")
print(f"Max month = {consolidated_df['year_month'].max()}")

Min month = 2010-04
Max month = 2024-08


### **Remove Partial Years from Data Set**

In [77]:
# Drop data from 2010 and 2024 as these are not full years
consolidated_df = consolidated_df [(consolidated_df['year_month'] >= '2011-01')\
    & ((consolidated_df['year_month'] < '2024-01'))].reset_index(drop = True)

# Check range of dates in year_month
print(f"Min month = {consolidated_df['year_month'].min()}")
print(f"Max month = {consolidated_df['year_month'].max()}")

Min month = 2011-01
Max month = 2023-12


In [79]:
# Insert new column for year
consolidated_df.insert(3, 'year', consolidated_df['year_month'].dt.strftime("%Y"))

print(consolidated_df.info())
consolidated_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145444 entries, 0 to 145443
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype    
---  ------        --------------   -----    
 0   borough_name  145444 non-null  object   
 1   major_cat     145444 non-null  object   
 2   minor_cat     145444 non-null  object   
 3   year          145444 non-null  object   
 4   year_month    145444 non-null  period[M]
 5   num_crimes    145444 non-null  int64    
dtypes: int64(1), object(4), period[M](1)
memory usage: 6.7+ MB
None


,borough_name,major_cat,minor_cat,year,year_month,num_crimes
0,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,2011-01,5
1,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,2011-02,12
2,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,2011-03,13
3,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,2011-04,12
4,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,2011-05,11


### **Validate Borough Names**

In [82]:
# Open csv file that contains unique borough names extracted from ONS data
b_names = pd.read_csv('london_boroughs_list.csv')

print(b_names.info())

# Check consolidated data borough names vs ONS data
print("Difference in borough names, 1 = consolidated data, 2 = ONS borough names\n")
diff_between_sets(consolidated_df['borough_name'], b_names['borough_name'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  33 non-null     object
 1   borough_name  33 non-null     object
dtypes: object(2)
memory usage: 660.0+ bytes
None
Difference in borough names, 1 = consolidated data, 2 = ONS borough names

In 1, Not In 2: {'Unknown', 'London Heathrow and London City Airports'}
In 2, Not In 1: {'City of London'}


**NOTES:**<br>
'London Heathrow and London City Airports' and Unknown' are not London Boroughs, therefore what should be done? 
 Delete?<br>
 'City of London' is a borough, but no crime has been recorded in it by MPS.

### **Delete Rows with Invalid Borough Names**

In [86]:
# View data with boroughs to be deleted
to_be_deleted = consolidated_df [(consolidated_df['borough_name'] == 'Unknown') |\
    (consolidated_df['borough_name'] == 'London Heathrow and London City Airports')].copy()

# Number of rows vs total
rows_all = consolidated_df.shape[0]
rows_tbd = to_be_deleted.shape[0]
pctr_tbd = rows_tbd / rows_all * 100

print(f"Summary:\n\n{rows_tbd} rows to be deleted out of {rows_all} total = {round(pctr_tbd, 2)}%")

# Number of crimes vs total
all_crime = consolidated_df['num_crimes'].sum()
tbd_crime = to_be_deleted['num_crimes'].sum()
pctc_tbd = tbd_crime / all_crime * 100

print(f"{tbd_crime} crimes to be deleted out of {all_crime} total = {round(pctc_tbd, 2)}%")

# By borough and by major cargory views
print("\n\nBy Borough:\n", to_be_deleted[['borough_name', 'num_crimes']]\
    .groupby('borough_name')\
    .sum())

print("\n\nBy Major Category:\n",to_be_deleted[['major_cat', 'num_crimes']]\
    .groupby('major_cat')\
    .sum())

Summary:

5748 rows to be deleted out of 145444 total = 3.95%
221504 crimes to be deleted out of 10452652 total = 2.12%


By Borough:
                                           num_crimes
borough_name                                        
London Heathrow and London City Airports        3714
Unknown                                       217790


By Major Category:
                                       num_crimes
major_cat                                       
Arson And Criminal Damage                   9277
Burglary                                    6905
Drug Offences                              18171
Fraud And Forgery                           1873
Miscellaneous Crimes Against Society        7681
Possession Of Weapons                       1785
Public Order Offences                      16474
Robbery                                     9544
Sexual Offences                            21561
Theft                                      56198
Vehicle Offences                           

**NOTES:**<br>
No guidance or insight from data source.  Heathrow and City airports are in different boroughs with a significant contribution in 2023 only.  Unknown borough is more problematic, but unable to include for borough to borough comparisons.  `Delete both from data set` but refer back and state what has been excluded.

In [89]:
# Remove rows with borough name 'London Heathrow and London City Airports' and Unknown'
print("Shape before = ", consolidated_df.shape)

consolidated_df = consolidated_df [(consolidated_df['borough_name'] != 'Unknown') &\
    (consolidated_df['borough_name'] != 'London Heathrow and London City Airports')]

# Validate change
print("Borough Names:\n", consolidated_df['borough_name'].unique())
print("Shape after = ", consolidated_df.shape)

Shape before =  (145444, 6)
Borough Names:
 ['Barking and Dagenham' 'Barnet' 'Bexley' 'Brent' 'Bromley' 'Camden'
 'Croydon' 'Ealing' 'Enfield' 'Greenwich' 'Hackney'
 'Hammersmith and Fulham' 'Haringey' 'Harrow' 'Havering' 'Hillingdon'
 'Hounslow' 'Islington' 'Kensington and Chelsea' 'Kingston upon Thames'
 'Lambeth' 'Lewisham' 'Merton' 'Newham' 'Redbridge' 'Richmond upon Thames'
 'Southwark' 'Sutton' 'Tower Hamlets' 'Waltham Forest' 'Wandsworth'
 'Westminster']
Shape after =  (139696, 6)


### **Add Borough Code to Data Set**

In [92]:
# Create a dictionary of values to assign to borough code column base
# on borough name
borough_dict = dict(zip(b_names.borough_name, b_names.borough_code))

# Insert new column and values
consolidated_df.insert(0, "borough_code",
                       consolidated_df['borough_name']\
                           .apply(lambda x: borough_dict[x] if x in borough_dict else "Unknown"),
                       True)

# View result
print(consolidated_df.info())
consolidated_df[['borough_code', 'borough_name']].groupby(['borough_code', 'borough_name']).count()

<class 'pandas.core.frame.DataFrame'>
Index: 139696 entries, 0 to 145443
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype    
---  ------        --------------   -----    
 0   borough_code  139696 non-null  object   
 1   borough_name  139696 non-null  object   
 2   major_cat     139696 non-null  object   
 3   minor_cat     139696 non-null  object   
 4   year          139696 non-null  object   
 5   year_month    139696 non-null  period[M]
 6   num_crimes    139696 non-null  int64    
dtypes: int64(1), object(5), period[M](1)
memory usage: 8.5+ MB
None


,
borough_code,borough_name
E09000002,Barking and Dagenham
E09000003,Barnet
E09000004,Bexley
E09000005,Brent
E09000006,Bromley
E09000007,Camden
E09000008,Croydon
E09000009,Ealing
E09000010,Enfield


### **Check For Gaps In Reporting**

In [95]:
consolidated_df.columns

Index(['borough_code', 'borough_name', 'major_cat', 'minor_cat', 'year',
       'year_month', 'num_crimes'],
      dtype='object')

In [97]:
# Check for gaps in data of major category by year
pivot_data = consolidated_df[['major_cat', 'year', 'num_crimes']].groupby(['major_cat', 'year']).sum().reset_index()
pivot_data.pivot(index = 'major_cat', columns = 'year', values = 'num_crimes')

year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
major_cat,,,,,,,,,,,,,
Arson And Criminal Damage,71847,60703,54785,57324,61038,62114,61621,56334,55519,49997,51798,51975,55886
Burglary,95657,93192,87197,75900,70503,68322,75996,80310,80376,60239,52339,51686,55363
Drug Offences,60242,51860,50317,44424,39834,39618,36556,36107,47473,54592,45390,43343,38125
Fraud And Forgery,21619,28541,2692,1,0,0,0,0,0,0,1,0,1
Miscellaneous Crimes Against Society,7708,7272,7728,8823,9980,11347,11194,11006,11145,11055,11121,11675,10058
Possession Of Weapons,4957,3817,3992,4323,4788,5847,7727,7688,7258,6727,5982,6276,6226
Public Order Offences,17058,21851,23362,32891,40449,43573,46224,46032,49859,51856,56635,54998,57800
Robbery,37595,34801,29026,22005,21383,22536,30952,32614,39123,26884,22023,26378,32664
Sexual Offences,9365,8891,9532,12590,14414,15995,17719,18157,18111,17179,21651,22856,22365


**NOTES:**<br>
`Fraud And Forgery` were transferred to a National Crime unit in 2014, therefore `delete both from data set` but refer back and state what has been excluded.

In [100]:
# Remove fraud and forgery from data set.
consolidated_df = consolidated_df [consolidated_df['major_cat'] != 'Fraud And Forgery']

# Check it worked
consolidated_df['major_cat'].unique()

array(['Arson And Criminal Damage', 'Burglary', 'Drug Offences',
       'Miscellaneous Crimes Against Society', 'Possession Of Weapons',
       'Public Order Offences', 'Robbery', 'Sexual Offences', 'Theft',
       'Vehicle Offences', 'Violence Against The Person'], dtype=object)

In [102]:
# Check for gaps in data of major category by year
pivot_data = consolidated_df[['minor_cat', 'year', 'num_crimes']].groupby(['minor_cat', 'year']).sum().reset_index()
pivot_data.pivot(index = 'minor_cat', columns = 'year', values = 'num_crimes')

year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
minor_cat,,,,,,,,,,,,,
Aggravated Vehicle Taking,798,651,668,860,859,930,1055,919,1059,892,651,717,657
Arson,2780,1917,1575,1941,2283,2189,2255,2000,1987,1758,1707,1851,2033
Bicycle Theft,22118,21448,18811,17210,16507,17997,21142,19345,19691,23428,20651,17603,15685
Burglary Business And Community,32160,31657,30783,27010,25845,25323,21535,20714,20913,14303,12769,15323,17846
Burglary In A Dwelling,63497,61535,56414,48890,44658,42999,54461,59596,59463,45936,39570,36363,37517
Criminal Damage,69067,58786,53210,55383,58755,59925,59366,54334,53532,48239,50091,50124,53853
Homicide,114,102,101,86,116,111,132,133,153,131,131,108,106
Interfering With A Motor Vehicle,4567,5107,4787,7585,10302,11443,13599,15001,16512,15006,10975,11005,10971
Misc Crimes Against Society,7708,7272,7728,8823,9980,11347,11194,11006,11145,11055,11121,11675,10058


**NOTES:**<br>
No further gaps found.

### **Create an Aggregated Data Set (Annual)** 

In [106]:
# Create an aggregated version of the data grouped by year
yearly_crime = consolidated_df[['borough_code', 'borough_name', 'major_cat', 'minor_cat', 'year', 'num_crimes']]\
    .groupby(['borough_code', 'borough_name', 'major_cat', 'minor_cat', 'year'])\
    .sum().reset_index()

yearly_crime

,borough_code,borough_name,major_cat,minor_cat,year,num_crimes
0,E09000002,Barking and Dagenham,Arson And Criminal Damage,Arson,2011,129
1,E09000002,Barking and Dagenham,Arson And Criminal Damage,Arson,2012,75
2,E09000002,Barking and Dagenham,Arson And Criminal Damage,Arson,2013,52
3,E09000002,Barking and Dagenham,Arson And Criminal Damage,Arson,2014,64
4,E09000002,Barking and Dagenham,Arson And Criminal Damage,Arson,2015,78
...,...,...,...,...,...,...
11223,E09000033,Westminster,Violence Against The Person,Violence Without Injury,2019,6794
11224,E09000033,Westminster,Violence Against The Person,Violence Without Injury,2020,5672
11225,E09000033,Westminster,Violence Against The Person,Violence Without Injury,2021,6436
11226,E09000033,Westminster,Violence Against The Person,Violence Without Injury,2022,6868


# **Prepare Tableau Data Sets**

## **London Boroughs**

In [110]:
# Create copy of boroughs, retaining required columns only
tab_borough_list = borough_list [borough_list['borough_name'] != 'City of London']

# View result.
print(tab_borough_list.info())
tab_borough_list.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 1 to 32
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  32 non-null     object
 1   borough_name  32 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes
None


,borough_code,borough_name
1,E09000002,Barking and Dagenham
2,E09000003,Barnet
3,E09000004,Bexley
4,E09000005,Brent
5,E09000006,Bromley
6,E09000007,Camden
7,E09000008,Croydon
8,E09000009,Ealing
9,E09000010,Enfield
10,E09000011,Greenwich


In [112]:
# Save clean data frame to new csv file without the index.
tab_borough_list.to_csv('tab_borough_list.csv', index = False)

# Test csv file imports correctly
# Import the data and view meta data and data.
test = pd.read_csv('tab_borough_list.csv')

print(test.info())
test.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  32 non-null     object
 1   borough_name  32 non-null     object
dtypes: object(2)
memory usage: 644.0+ bytes
None


,borough_code,borough_name
0,E09000002,Barking and Dagenham
1,E09000003,Barnet
2,E09000004,Bexley
3,E09000005,Brent
4,E09000006,Bromley
5,E09000007,Camden
6,E09000008,Croydon
7,E09000009,Ealing
8,E09000010,Enfield
9,E09000011,Greenwich


## **London Borough Population by Year**

In [115]:
# Create copy of boroughs, retaining required columns only
tab_population = boroughs[['borough_code', 'year', 'population']].copy()

# Year needs to be stored a a date for best use in Tableau, ie 1st Jan each year.
tab_population['year'] = pd.to_datetime(tab_population['year'], format='%Y')
tab_population.rename(columns = {'year' : 'year_asdate'}, inplace = True)

# View result.
print(tab_population.info())
tab_population.head()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 12 to 1688
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   borough_code  429 non-null    object        
 1   year_asdate   429 non-null    datetime64[ns]
 2   population    429 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 13.4+ KB
None


,borough_code,year_asdate,population
12,E09000001,2011-01-01,7412
13,E09000001,2012-01-01,7204
14,E09000001,2013-01-01,6848
15,E09000001,2014-01-01,6872
16,E09000001,2015-01-01,7160


In [117]:
# Save clean data frame to new csv file without the index.
tab_population.to_csv('tab_borough_population.csv', index = False)

# Test csv file imports correctly
# Import the data and view meta data and data.
test = pd.read_csv('tab_borough_population.csv')

# Convert year from int to str
#test['year'] = test['year'].apply(str)

print(test.info())
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  429 non-null    object
 1   year_asdate   429 non-null    object
 2   population    429 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 10.2+ KB
None


,borough_code,year_asdate,population
0,E09000001,2011-01-01,7412
1,E09000001,2012-01-01,7204
2,E09000001,2013-01-01,6848
3,E09000001,2014-01-01,6872
4,E09000001,2015-01-01,7160


## **Crime by London Borough, Year, Major and Minor Crime Category**

In [120]:
# Create copy of boroughs, retaining required columns only
tab_crime = yearly_crime[['borough_code', 'year', 'major_cat', 'minor_cat', 'num_crimes']].copy()

# Year needs to be stored a a date for best use in Tableau, ie 1st Jan each year.
tab_crime['year'] = pd.to_datetime(tab_crime['year'], format='%Y')
tab_crime.rename(columns = {'year' : 'year_asdate'}, inplace = True)

# View result.
print(tab_crime.info())
tab_crime.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11228 entries, 0 to 11227
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   borough_code  11228 non-null  object        
 1   year_asdate   11228 non-null  datetime64[ns]
 2   major_cat     11228 non-null  object        
 3   minor_cat     11228 non-null  object        
 4   num_crimes    11228 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 438.7+ KB
None


,borough_code,year_asdate,major_cat,minor_cat,num_crimes
0,E09000002,2011-01-01,Arson And Criminal Damage,Arson,129
1,E09000002,2012-01-01,Arson And Criminal Damage,Arson,75
2,E09000002,2013-01-01,Arson And Criminal Damage,Arson,52
3,E09000002,2014-01-01,Arson And Criminal Damage,Arson,64
4,E09000002,2015-01-01,Arson And Criminal Damage,Arson,78


In [122]:
# Save clean data frame to new csv file without the index.
tab_crime.to_csv('tab_yearly_crime.csv', index = False)

# Test csv file imports correctly
# Import the data and view meta data and data.
test = pd.read_csv('tab_yearly_crime.csv')

# Convert year from int to str
#test['year'] = test['year'].apply(str)

print(test.info())
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11228 entries, 0 to 11227
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   borough_code  11228 non-null  object
 1   year_asdate   11228 non-null  object
 2   major_cat     11228 non-null  object
 3   minor_cat     11228 non-null  object
 4   num_crimes    11228 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 438.7+ KB
None


,borough_code,year_asdate,major_cat,minor_cat,num_crimes
0,E09000002,2011-01-01,Arson And Criminal Damage,Arson,129
1,E09000002,2012-01-01,Arson And Criminal Damage,Arson,75
2,E09000002,2013-01-01,Arson And Criminal Damage,Arson,52
3,E09000002,2014-01-01,Arson And Criminal Damage,Arson,64
4,E09000002,2015-01-01,Arson And Criminal Damage,Arson,78
